# Module 3 - Exercise 2: Tree Algorithms

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jumpingsphinx/jumpingsphinx.github.io/blob/main/notebooks/module3-trees/exercise2-tree-algorithms.ipynb)

## Learning Objectives

By the end of this exercise, you will be able to:

- Implement the ID3 algorithm from scratch for categorical data
- Apply ID3 to the classic Play Tennis dataset
- Calculate and compare Information Gain and Gain Ratio (C4.5 improvement)
- Handle continuous features using discretization techniques
- Use CART algorithm with sklearn on real datasets
- Compare ID3 and CART algorithms
- Handle missing values in decision trees
- Apply tree algorithms to real-world classification problems
- Understand trade-offs between different tree algorithms

## Prerequisites

- Understanding of entropy and information theory
- Basic probability and statistics
- Familiarity with classification problems
- NumPy proficiency

## Setup

Run this cell first to import required libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.datasets import load_wine, load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import KBinsDiscretizer
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Setup complete!")

---

## Part 1: ID3 Algorithm Implementation

### Background

ID3 (Iterative Dichotomiser 3) is one of the earliest and most fundamental decision tree algorithms. It uses **Information Gain** to select the best attribute at each node.

**Key Concepts:**

1. **Entropy**: Measures impurity/disorder in a dataset
   $$H(S) = -\sum_{i=1}^{c} p_i \log_2(p_i)$$
   where $p_i$ is the proportion of class $i$

2. **Information Gain**: Reduction in entropy after splitting
   $$IG(S, A) = H(S) - \sum_{v \in Values(A)} \frac{|S_v|}{|S|} H(S_v)$$
   where $S_v$ is the subset where attribute $A$ has value $v$

3. **Algorithm Steps**:
   - Calculate entropy of the dataset
   - For each attribute, calculate information gain
   - Split on attribute with highest information gain
   - Recursively build tree until stopping criterion

### Exercise 1.1: Implement Entropy Function

**Task:** Implement entropy calculation from scratch.

In [ ]:
def entropy(y):
    """
    Calculate entropy of a dataset.
    
    Parameters:
    -----------
    y : array-like
        Target values
    
    Returns:
    --------
    float
        Entropy value (0 = pure, higher = more impure)
    """
    # Your code here
    # Hint: Count class frequencies, calculate probabilities
    # Handle edge case: log(0) is undefined, but 0*log(0) should be 0
    
    if len(y) == 0:
        return 0
    
    # Count occurrences of each class
    counts = 
    
    # Calculate probabilities
    probabilities = 
    
    # Calculate entropy: -Σ p_i * log2(p_i)
    ent = 0
    for p in probabilities:
        if p > 0:  # Avoid log(0)
            ent += 
    
    return ent

# Test entropy function
print("Testing Entropy Function:")
print(f"Entropy of [0,0,0,0]: {entropy([0,0,0,0]):.4f} (should be 0.0000 - pure)")
print(f"Entropy of [0,0,1,1]: {entropy([0,0,1,1]):.4f} (should be 1.0000 - max impurity)")
print(f"Entropy of [0,0,0,1]: {entropy([0,0,0,1]):.4f} (should be 0.8113)")
print(f"Entropy of [0,1,2]: {entropy([0,1,2]):.4f} (should be 1.5850 - 3 classes)")

assert abs(entropy([0,0,0,0]) - 0.0) < 0.001, "Pure set should have 0 entropy"
assert abs(entropy([0,0,1,1]) - 1.0) < 0.001, "50-50 split should have entropy 1"
print("\n✓ Entropy function works!")

### Exercise 1.2: Implement Information Gain

**Task:** Calculate information gain for a given attribute.

In [ ]:
def information_gain(X, y, attribute_index):
    """
    Calculate information gain for a given attribute.
    
    IG(S, A) = H(S) - Σ (|S_v| / |S|) * H(S_v)
    
    Parameters:
    -----------
    X : pd.DataFrame or np.ndarray
        Features
    y : array-like
        Target values
    attribute_index : int or str
        Index or name of attribute to evaluate
    
    Returns:
    --------
    float
        Information gain
    """
    # Your code here
    # Calculate initial entropy
    total_entropy = 
    
    # Get attribute values
    if isinstance(X, pd.DataFrame):
        attribute_values = X.iloc[:, attribute_index] if isinstance(attribute_index, int) else X[attribute_index]
    else:
        attribute_values = X[:, attribute_index]
    
    # Calculate weighted average of entropies after split
    weighted_entropy = 0
    unique_values = np.unique(attribute_values)
    
    for value in unique_values:
        # Get subset where attribute == value
        mask = 
        subset_y = 
        
        # Calculate weight and entropy for this subset
        weight = 
        subset_entropy = 
        
        weighted_entropy += weight * subset_entropy
    
    # Information gain = total entropy - weighted entropy
    ig = 
    
    return ig

# Test with simple example
X_test = pd.DataFrame({
    'Weather': ['Sunny', 'Sunny', 'Rainy', 'Rainy'],
    'Temp': ['Hot', 'Cool', 'Cool', 'Cool']
})
y_test = np.array([0, 0, 1, 1])

ig_weather = information_gain(X_test, y_test, 'Weather')
ig_temp = information_gain(X_test, y_test, 'Temp')

print("Testing Information Gain:")
print(f"IG(Weather): {ig_weather:.4f}")
print(f"IG(Temp): {ig_temp:.4f}")
print("\n✓ Information gain function works!")

### Exercise 1.3: Implement ID3 Algorithm

**Task:** Build a complete ID3 decision tree classifier.

In [ ]:
class ID3Node:
    """Node in an ID3 decision tree."""
    def __init__(self, attribute=None, label=None, branches=None):
        self.attribute = attribute  # Attribute to split on (None for leaf)
        self.label = label          # Class label (for leaf nodes)
        self.branches = branches or {}  # Dictionary: value -> child node
    
    def is_leaf(self):
        return self.label is not None


class ID3Classifier:
    """
    ID3 Decision Tree Classifier for categorical features.
    """
    def __init__(self, max_depth=None, min_samples_split=2):
        """
        Parameters:
        -----------
        max_depth : int or None
            Maximum depth of tree
        min_samples_split : int
            Minimum samples required to split
        """
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None
        self.feature_names = None
    
    def fit(self, X, y, feature_names=None):
        """
        Build ID3 decision tree.
        
        Parameters:
        -----------
        X : pd.DataFrame or np.ndarray
            Training features (categorical)
        y : array-like
            Target values
        feature_names : list, optional
            Names of features
        """
        if isinstance(X, pd.DataFrame):
            self.feature_names = X.columns.tolist()
            X = X.values
        else:
            self.feature_names = feature_names or [f"Feature_{i}" for i in range(X.shape[1])]
        
        y = np.array(y)
        
        # Build tree recursively
        available_attributes = list(range(X.shape[1]))
        self.tree = self._build_tree(X, y, available_attributes, depth=0)
        
        return self
    
    def _build_tree(self, X, y, available_attributes, depth):
        """
        Recursively build the decision tree.
        
        Returns:
        --------
        ID3Node
        """
        # Your code here
        
        # Base case 1: All samples have same class
        if len(np.unique(y)) == 1:
            return ID3Node(label=y[0])
        
        # Base case 2: No more attributes or max depth reached
        if len(available_attributes) == 0 or (self.max_depth and depth >= self.max_depth):
            # Return most common class
            most_common = 
            return ID3Node(label=most_common)
        
        # Base case 3: Too few samples to split
        if len(y) < self.min_samples_split:
            most_common = 
            return ID3Node(label=most_common)
        
        # Find best attribute using information gain
        best_attribute = None
        best_gain = -1
        
        for attr in available_attributes:
            gain = information_gain(X, y, attr)
            if gain > best_gain:
                best_gain = gain
                best_attribute = attr
        
        # If no information gain, return leaf
        if best_gain == 0:
            most_common = 
            return ID3Node(label=most_common)
        
        # Create node for this attribute
        node = ID3Node(attribute=best_attribute)
        
        # Get unique values for this attribute
        unique_values = np.unique(X[:, best_attribute])
        
        # Remove this attribute from available list
        remaining_attributes = [a for a in available_attributes if a != best_attribute]
        
        # Create branches for each value
        for value in unique_values:
            # Get subset where attribute == value
            mask = X[:, best_attribute] == value
            X_subset = X[mask]
            y_subset = y[mask]
            
            # Recursively build subtree
            if len(y_subset) > 0:
                node.branches[value] = self._build_tree(
                    X_subset, y_subset, remaining_attributes, depth + 1
                )
            else:
                # If no samples, use most common class from parent
                most_common = 
                node.branches[value] = ID3Node(label=most_common)
        
        return node
    
    def predict(self, X):
        """
        Predict class labels.
        
        Parameters:
        -----------
        X : pd.DataFrame or np.ndarray
            Test features
        
        Returns:
        --------
        np.ndarray
            Predicted labels
        """
        if isinstance(X, pd.DataFrame):
            X = X.values
        
        return np.array([self._predict_one(x) for x in X])
    
    def _predict_one(self, x):
        """Predict label for a single sample."""
        node = self.tree
        
        while not node.is_leaf():
            attribute_value = x[node.attribute]
            
            # If value not seen in training, return most common class
            if attribute_value not in node.branches:
                # Find a leaf and return its label
                # (Simple fallback - could be improved)
                break
            
            node = node.branches[attribute_value]
        
        return node.label
    
    def print_tree(self, node=None, depth=0, value=None):
        """
        Print the decision tree in a readable format.
        """
        if node is None:
            node = self.tree
        
        indent = "  " * depth
        
        if value is not None:
            print(f"{indent}└─ [{value}]")
        
        if node.is_leaf():
            print(f"{indent}   → Class: {node.label}")
        else:
            attr_name = self.feature_names[node.attribute]
            print(f"{indent}   Split on: {attr_name}")
            for value, child in sorted(node.branches.items()):
                self.print_tree(child, depth + 1, value)

print("ID3 Classifier implemented!")

---

## Part 2: Play Tennis Dataset

### Background

The Play Tennis dataset is a classic example used to teach decision trees. It predicts whether someone will play tennis based on weather conditions.

**Features:**
- Outlook: Sunny, Overcast, Rainy
- Temperature: Hot, Mild, Cool
- Humidity: High, Normal
- Wind: Weak, Strong

**Target:** Play Tennis (Yes/No)

### Exercise 2.1: Apply ID3 to Play Tennis Dataset

**Task:** Build a decision tree for the Play Tennis dataset.

In [ ]:
# Create Play Tennis dataset
play_tennis_data = {
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Rainy', 'Overcast',
                'Sunny', 'Sunny', 'Rainy', 'Sunny', 'Overcast', 'Overcast', 'Rainy'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool',
                    'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal',
                 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
    'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong',
             'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Strong'],
    'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes',
             'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
}

df_tennis = pd.DataFrame(play_tennis_data)

print("Play Tennis Dataset:")
print(df_tennis)
print(f"\nDataset shape: {df_tennis.shape}")
print(f"\nClass distribution:")
print(df_tennis['Play'].value_counts())

In [ ]:
# Calculate information gain for each attribute
X_tennis = df_tennis.drop('Play', axis=1)
y_tennis = df_tennis['Play']

print("Information Gain for Each Attribute:\n")
for col in X_tennis.columns:
    ig = information_gain(X_tennis, y_tennis, col)
    print(f"{col:15s}: {ig:.4f}")

print("\nBest attribute to split on: The one with highest IG!")

In [ ]:
# Build ID3 tree
id3_tennis = ID3Classifier(max_depth=5)
id3_tennis.fit(X_tennis, y_tennis)

# Print the tree
print("\nID3 Decision Tree for Play Tennis:\n")
print("=" * 50)
id3_tennis.print_tree()
print("=" * 50)

In [ ]:
# Test predictions on training data
y_pred_tennis = id3_tennis.predict(X_tennis)
accuracy_tennis = accuracy_score(y_tennis, y_pred_tennis)

print(f"\nTraining Accuracy: {accuracy_tennis:.4f}")
print("\nPredictions vs Actual:")
comparison = pd.DataFrame({
    'Actual': y_tennis,
    'Predicted': y_pred_tennis
})
print(comparison)

# Test on new examples
print("\nTesting on new examples:")
new_examples = pd.DataFrame({
    'Outlook': ['Sunny', 'Rainy', 'Overcast'],
    'Temperature': ['Cool', 'Mild', 'Hot'],
    'Humidity': ['Normal', 'Normal', 'High'],
    'Wind': ['Weak', 'Weak', 'Weak']
})

predictions = id3_tennis.predict(new_examples)
for i, pred in enumerate(predictions):
    print(f"Example {i+1}: {dict(new_examples.iloc[i])} → {pred}")

---

## Part 3: Gain Ratio (C4.5 Improvement)

### Background

**Problem with Information Gain:** It is biased toward attributes with many values.

**C4.5's Solution: Gain Ratio**

$$\text{GainRatio}(S, A) = \frac{IG(S, A)}{\text{SplitInfo}(S, A)}$$

Where **Split Information** measures how much information is needed to specify the value of an attribute:

$$\text{SplitInfo}(S, A) = -\sum_{v \in \text{Values}(A)} \frac{|S_v|}{|S|} \log_2 \frac{|S_v|}{|S|}$$

This penalizes attributes that split data into many small subsets.

### Exercise 3.1: Implement Gain Ratio

**Task:** Implement split information and gain ratio.

In [ ]:
def split_information(X, attribute_index):
    """
    Calculate split information for an attribute.
    
    SplitInfo(S, A) = -Σ (|S_v| / |S|) * log2(|S_v| / |S|)
    
    Parameters:
    -----------
    X : pd.DataFrame or np.ndarray
        Features
    attribute_index : int or str
        Attribute to evaluate
    
    Returns:
    --------
    float
        Split information
    """
    # Your code here
    if isinstance(X, pd.DataFrame):
        attribute_values = X.iloc[:, attribute_index] if isinstance(attribute_index, int) else X[attribute_index]
    else:
        attribute_values = X[:, attribute_index]
    
    n_total = len(attribute_values)
    value_counts = pd.Series(attribute_values).value_counts()
    
    split_info = 0
    for count in value_counts:
        proportion = count / n_total
        if proportion > 0:
            split_info += 
    
    return split_info


def gain_ratio(X, y, attribute_index):
    """
    Calculate gain ratio (C4.5 criterion).
    
    GainRatio = InformationGain / SplitInformation
    
    Parameters:
    -----------
    X : pd.DataFrame or np.ndarray
        Features
    y : array-like
        Target values
    attribute_index : int or str
        Attribute to evaluate
    
    Returns:
    --------
    float
        Gain ratio
    """
    # Your code here
    ig = 
    si = 
    
    # Avoid division by zero
    if si == 0:
        return 0
    
    gr = 
    return gr

print("Gain Ratio functions implemented!")

### Exercise 3.2: Compare Information Gain vs Gain Ratio

**Task:** Compare both metrics on the Play Tennis dataset.

In [ ]:
# Calculate both metrics for each attribute
print("Comparison: Information Gain vs Gain Ratio\n")
print(f"{'Attribute':<15} {'Info Gain':<12} {'Split Info':<12} {'Gain Ratio':<12}")
print("="*55)

for col in X_tennis.columns:
    ig = information_gain(X_tennis, y_tennis, col)
    si = split_information(X_tennis, col)
    gr = gain_ratio(X_tennis, y_tennis, col)
    print(f"{col:<15} {ig:<12.4f} {si:<12.4f} {gr:<12.4f}")

print("\nObservation: Gain ratio normalizes by split information,")
print("reducing bias toward attributes with many values.")

### Exercise 3.3: Demonstrate Bias with Synthetic Example

**Task:** Create an example showing information gain's bias toward many-valued attributes.

In [ ]:
# Create dataset with an attribute having unique values (like ID)
n_samples = 20
bias_data = pd.DataFrame({
    'ID': list(range(n_samples)),  # Unique for each sample
    'UsefulFeature': ['A']*10 + ['B']*10,  # Actually correlated with target
    'RandomFeature': np.random.choice(['X', 'Y', 'Z'], n_samples),
})
# Target: correlated with UsefulFeature
bias_target = np.array([0]*8 + [1]*2 + [1]*8 + [0]*2)

print("Dataset with potential bias:")
print(bias_data.head(10))
print(f"\nTarget: {bias_target[:10]}...")

print("\n" + "="*55)
print(f"{'Attribute':<20} {'Info Gain':<15} {'Gain Ratio':<15}")
print("="*55)

for col in bias_data.columns:
    ig = information_gain(bias_data, bias_target, col)
    gr = gain_ratio(bias_data, bias_target, col)
    print(f"{col:<20} {ig:<15.4f} {gr:<15.4f}")

print("\nNotice: ID has high Information Gain (splits perfectly!)")
print("But low Gain Ratio (penalty for many unique values).")
print("This shows why C4.5 uses Gain Ratio instead of Information Gain.")

---

## Part 4: Handling Continuous Features

### Background

ID3 was designed for categorical features. C4.5 extends it to handle continuous features by:

1. **Discretization**: Convert continuous values into bins/categories
2. **Binary splits**: Find best threshold to split into two groups

**Methods:**
- Equal-width binning
- Equal-frequency binning
- Entropy-based binning

### Exercise 4.1: Discretize Continuous Features

**Task:** Implement and compare discretization strategies.

In [ ]:
# Create dataset with continuous features
from sklearn.datasets import make_classification

X_cont, y_cont = make_classification(
    n_samples=100, n_features=2, n_informative=2,
    n_redundant=0, n_clusters_per_class=1, random_state=42
)

# Visualize continuous data
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.scatter(X_cont[:, 0], X_cont[:, 1], c=y_cont, cmap='viridis', alpha=0.6)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Original Continuous Data')
plt.colorbar(label='Class')

# Equal-width binning
discretizer_width = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
X_width = discretizer_width.fit_transform(X_cont)

plt.subplot(1, 3, 2)
plt.scatter(X_width[:, 0], X_width[:, 1], c=y_cont, cmap='viridis', alpha=0.6)
plt.xlabel('Feature 1 (binned)')
plt.ylabel('Feature 2 (binned)')
plt.title('Equal-Width Binning (3 bins)')
plt.colorbar(label='Class')

# Equal-frequency binning
discretizer_freq = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
X_freq = discretizer_freq.fit_transform(X_cont)

plt.subplot(1, 3, 3)
plt.scatter(X_freq[:, 0], X_freq[:, 1], c=y_cont, cmap='viridis', alpha=0.6)
plt.xlabel('Feature 1 (binned)')
plt.ylabel('Feature 2 (binned)')
plt.title('Equal-Frequency Binning (3 bins)')
plt.colorbar(label='Class')

plt.tight_layout()
plt.show()

print("Discretization strategies compared!")

In [ ]:
# Convert to categorical and apply ID3
X_discrete = pd.DataFrame(
    X_width.astype(int),
    columns=['Feature1_bin', 'Feature2_bin']
)

# Train ID3 on discretized data
id3_discrete = ID3Classifier(max_depth=3)
id3_discrete.fit(X_discrete, y_cont)

# Evaluate
y_pred_discrete = id3_discrete.predict(X_discrete)
accuracy_discrete = accuracy_score(y_cont, y_pred_discrete)

print(f"ID3 on Discretized Data:")
print(f"Accuracy: {accuracy_discrete:.4f}")

print("\nDecision Tree:")
id3_discrete.print_tree()

---

## Part 5: CART Algorithm

### Background

**CART (Classification and Regression Trees)** differs from ID3/C4.5:

| Feature | ID3/C4.5 | CART |
|---------|----------|------|
| Split criterion | Information Gain / Gain Ratio | Gini Impurity |
| Splits | Multi-way | Binary only |
| Continuous features | Discretization (C4.5) | Native support |
| Pruning | Post-pruning | Cost-complexity |
| Tasks | Classification | Classification & Regression |

**Gini Impurity:**
$$\text{Gini}(S) = 1 - \sum_{i=1}^{c} p_i^2$$

### Exercise 5.1: Apply CART to Wine Dataset

**Task:** Use sklearn's DecisionTreeClassifier (CART) on the Wine dataset.

In [ ]:
# Load Wine dataset
wine = load_wine()
X_wine = wine.data
y_wine = wine.target

print("Wine Dataset:")
print(f"Shape: {X_wine.shape}")
print(f"Features: {wine.feature_names}")
print(f"Classes: {wine.target_names}")
print(f"\nClass distribution:")
print(pd.Series(y_wine).value_counts().sort_index())

# Split data
X_train_wine, X_test_wine, y_train_wine, y_test_wine = train_test_split(
    X_wine, y_wine, test_size=0.3, random_state=42, stratify=y_wine
)

print(f"\nTrain set: {X_train_wine.shape}")
print(f"Test set: {X_test_wine.shape}")

In [ ]:
# Train CART with Gini criterion
cart_gini = DecisionTreeClassifier(
    criterion='gini',
    max_depth=5,
    min_samples_split=5,
    random_state=42
)
cart_gini.fit(X_train_wine, y_train_wine)

# Train CART with Entropy criterion
cart_entropy = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=5,
    min_samples_split=5,
    random_state=42
)
cart_entropy.fit(X_train_wine, y_train_wine)

# Evaluate both
y_pred_gini = cart_gini.predict(X_test_wine)
y_pred_entropy = cart_entropy.predict(X_test_wine)

acc_gini = accuracy_score(y_test_wine, y_pred_gini)
acc_entropy = accuracy_score(y_test_wine, y_pred_entropy)

print("CART Performance on Wine Dataset:\n")
print(f"Gini Criterion:    {acc_gini:.4f}")
print(f"Entropy Criterion: {acc_entropy:.4f}")

# Detailed metrics for Gini
print("\nDetailed Classification Report (Gini):")
print(classification_report(y_test_wine, y_pred_gini, target_names=wine.target_names))

In [ ]:
# Visualize the tree
plt.figure(figsize=(20, 10))
plot_tree(
    cart_gini,
    feature_names=wine.feature_names,
    class_names=wine.target_names,
    filled=True,
    fontsize=10,
    rounded=True
)
plt.title('CART Decision Tree on Wine Dataset (Gini)', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance
feature_importance = cart_gini.feature_importances_
indices = np.argsort(feature_importance)[::-1]

plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_importance)), feature_importance[indices])
plt.xticks(range(len(feature_importance)), [wine.feature_names[i] for i in indices], rotation=45, ha='right')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importance in CART (Wine Dataset)')
plt.tight_layout()
plt.show()

print("Top 5 Most Important Features:")
for i in range(5):
    idx = indices[i]
    print(f"{i+1}. {wine.feature_names[idx]}: {feature_importance[idx]:.4f}")

### Exercise 5.2: Tune CART Hyperparameters

**Task:** Experiment with different hyperparameters to find the best tree.

In [ ]:
# Your turn: Try different hyperparameters
# Test different max_depth values

depths = [2, 3, 5, 7, 10, None]
train_scores = []
test_scores = []

for depth in depths:
    # Your code here
    cart = DecisionTreeClassifier(
        max_depth=depth,
        random_state=42
    )
    cart.fit(X_train_wine, y_train_wine)
    
    train_acc = 
    test_acc = 
    
    train_scores.append(train_acc)
    test_scores.append(test_acc)

# Plot results
depth_labels = [str(d) if d is not None else 'None' for d in depths]

plt.figure(figsize=(10, 6))
plt.plot(depth_labels, train_scores, 'o-', label='Train', linewidth=2, markersize=8)
plt.plot(depth_labels, test_scores, 's-', label='Test', linewidth=2, markersize=8)
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.title('CART Performance vs Tree Depth')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim([0.7, 1.05])
plt.show()

best_depth_idx = np.argmax(test_scores)
print(f"\nBest max_depth: {depths[best_depth_idx]}")
print(f"Test accuracy: {test_scores[best_depth_idx]:.4f}")

---

## Part 6: Algorithm Comparison

### Exercise 6.1: Compare ID3 and CART

**Task:** Compare ID3 (after discretization) with CART on the same dataset.

In [ ]:
# Prepare Wine data for ID3 (discretize features)
discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
X_wine_discrete = discretizer.fit_transform(X_wine)

# Split discretized data
X_train_disc, X_test_disc, y_train_disc, y_test_disc = train_test_split(
    X_wine_discrete, y_wine, test_size=0.3, random_state=42, stratify=y_wine
)

# Train ID3 on discretized data
id3_wine = ID3Classifier(max_depth=5, min_samples_split=5)
id3_wine.fit(X_train_disc.astype(int), y_train_disc, feature_names=wine.feature_names)

# Predict
y_pred_id3 = id3_wine.predict(X_test_disc.astype(int))
acc_id3 = accuracy_score(y_test_disc, y_pred_id3)

# Train CART on original continuous data
cart_wine = DecisionTreeClassifier(max_depth=5, min_samples_split=5, random_state=42)
cart_wine.fit(X_train_wine, y_train_wine)
y_pred_cart = cart_wine.predict(X_test_wine)
acc_cart = accuracy_score(y_test_wine, y_pred_cart)

# Compare
print("Algorithm Comparison on Wine Dataset:\n")
print("="*60)
print(f"{'Algorithm':<30} {'Accuracy':<15} {'Notes':<15}")
print("="*60)
print(f"{'ID3 (discretized)':<30} {acc_id3:<15.4f} {'5 bins/feature'}")
print(f"{'CART (Gini)':<30} {acc_cart:<15.4f} {'Native continuous'}")
print("="*60)

print("\nKey Differences:")
print("1. ID3 requires discretization for continuous features")
print("2. CART handles continuous features natively")
print("3. ID3 uses multi-way splits, CART uses binary splits")
print("4. ID3 uses Information Gain, CART uses Gini/Entropy")

---

## Part 7: Handling Missing Values

### Background

Real-world data often has missing values. Decision trees can handle them through:

1. **Surrogate splits**: Find backup splits that mimic the primary split
2. **Probabilistic splits**: Send sample down multiple branches with weights
3. **Imputation**: Fill missing values before building tree

### Exercise 7.1: Handle Missing Values

**Task:** Compare strategies for handling missing data.

In [ ]:
# Create Wine dataset with missing values
X_wine_missing = X_wine.copy()
y_wine_missing = y_wine.copy()

# Randomly set 10% of values to NaN
np.random.seed(42)
missing_mask = np.random.random(X_wine_missing.shape) < 0.1
X_wine_missing[missing_mask] = np.nan

print(f"Created dataset with missing values")
print(f"Total missing values: {np.isnan(X_wine_missing).sum()}")
print(f"Percentage missing: {np.isnan(X_wine_missing).sum() / X_wine_missing.size * 100:.2f}%")

# Split
X_train_miss, X_test_miss, y_train_miss, y_test_miss = train_test_split(
    X_wine_missing, y_wine_missing, test_size=0.3, random_state=42, stratify=y_wine_missing
)

In [ ]:
from sklearn.impute import SimpleImputer

# Strategy 1: Mean imputation
imputer_mean = SimpleImputer(strategy='mean')
X_train_mean = imputer_mean.fit_transform(X_train_miss)
X_test_mean = imputer_mean.transform(X_test_miss)

cart_mean = DecisionTreeClassifier(max_depth=5, random_state=42)
cart_mean.fit(X_train_mean, y_train_miss)
acc_mean = cart_mean.score(X_test_mean, y_test_miss)

# Strategy 2: Median imputation
imputer_median = SimpleImputer(strategy='median')
X_train_median = imputer_median.fit_transform(X_train_miss)
X_test_median = imputer_median.transform(X_test_miss)

cart_median = DecisionTreeClassifier(max_depth=5, random_state=42)
cart_median.fit(X_train_median, y_train_miss)
acc_median = cart_median.score(X_test_median, y_test_miss)

# Strategy 3: Most frequent (for comparison)
imputer_freq = SimpleImputer(strategy='most_frequent')
X_train_freq = imputer_freq.fit_transform(X_train_miss)
X_test_freq = imputer_freq.transform(X_test_miss)

cart_freq = DecisionTreeClassifier(max_depth=5, random_state=42)
cart_freq.fit(X_train_freq, y_train_miss)
acc_freq = cart_freq.score(X_test_freq, y_test_miss)

# Compare
print("Missing Value Handling Strategies:\n")
print("="*50)
print(f"{'Strategy':<25} {'Test Accuracy':<15}")
print("="*50)
print(f"{'Mean Imputation':<25} {acc_mean:<15.4f}")
print(f"{'Median Imputation':<25} {acc_median:<15.4f}")
print(f"{'Most Frequent Imputation':<25} {acc_freq:<15.4f}")
print("="*50)

print("\nNote: sklearn's DecisionTreeClassifier doesn't natively support")
print("missing values. We must impute them before training.")
print("For native missing value support, consider XGBoost or LightGBM.")

---

## Part 8: Real-World Application - Breast Cancer Classification

### Exercise 8.1: Full Pipeline on Breast Cancer Dataset

**Task:** Apply everything you've learned to a medical diagnosis problem.

In [ ]:
# Load Breast Cancer dataset
cancer = load_breast_cancer()
X_cancer = cancer.data
y_cancer = cancer.target

print("Breast Cancer Dataset:")
print(f"Shape: {X_cancer.shape}")
print(f"Classes: {cancer.target_names}")
print(f"  0 = {cancer.target_names[0]}")
print(f"  1 = {cancer.target_names[1]}")
print(f"\nClass distribution:")
print(pd.Series(y_cancer).value_counts())
print(f"\nFeatures (first 10):")
for i, name in enumerate(cancer.feature_names[:10]):
    print(f"  {i}: {name}")
print(f"  ... ({len(cancer.feature_names)} total features)")

In [ ]:
# Split data
X_train_cancer, X_test_cancer, y_train_cancer, y_test_cancer = train_test_split(
    X_cancer, y_cancer, test_size=0.2, random_state=42, stratify=y_cancer
)

print(f"Train set: {X_train_cancer.shape}")
print(f"Test set: {X_test_cancer.shape}")

# Your turn: Build and evaluate CART model
# Try to achieve >90% accuracy
# Experiment with hyperparameters!

# Your code here
cart_cancer = DecisionTreeClassifier(
    criterion=,  # Try 'gini' or 'entropy'
    max_depth=,  # Experiment with depth
    min_samples_split=,
    min_samples_leaf=,
    random_state=42
)

cart_cancer.fit()

# Predictions
y_pred_cancer = 

# Evaluate
acc_cancer = accuracy_score(y_test_cancer, y_pred_cancer)

print(f"\nTest Accuracy: {acc_cancer:.4f}")
print("\nClassification Report:")
print(classification_report(y_test_cancer, y_pred_cancer, target_names=cancer.target_names))

# Confusion matrix
cm = confusion_matrix(y_test_cancer, y_pred_cancer)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=cancer.target_names,
            yticklabels=cancer.target_names)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix - Breast Cancer Classification')
plt.show()

print("\nConfusion Matrix Interpretation:")
print(f"True Negatives (malignant correctly classified): {cm[0,0]}")
print(f"False Positives (malignant predicted as benign): {cm[0,1]}")
print(f"False Negatives (benign predicted as malignant): {cm[1,0]}")
print(f"True Positives (benign correctly classified): {cm[1,1]}")

In [ ]:
# Feature importance for interpretation
feature_importance_cancer = cart_cancer.feature_importances_
indices = np.argsort(feature_importance_cancer)[::-1][:10]  # Top 10

plt.figure(figsize=(12, 6))
plt.barh(range(len(indices)), feature_importance_cancer[indices])
plt.yticks(range(len(indices)), [cancer.feature_names[i] for i in indices])
plt.xlabel('Importance')
plt.title('Top 10 Most Important Features for Breast Cancer Classification')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 5 Most Important Features:")
for i in range(5):
    idx = indices[i]
    print(f"{i+1}. {cancer.feature_names[idx]}: {feature_importance_cancer[idx]:.4f}")

print("\nMedical Interpretation:")
print("These features are most useful for distinguishing malignant from benign tumors.")
print("Doctors can focus on these characteristics during diagnosis.")

In [ ]:
# Cross-validation for robust evaluation
cv_scores = cross_val_score(cart_cancer, X_cancer, y_cancer, cv=5)

print("5-Fold Cross-Validation Results:")
print(f"Fold scores: {cv_scores}")
print(f"Mean accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

plt.figure(figsize=(8, 5))
plt.boxplot([cv_scores])
plt.ylabel('Accuracy')
plt.title('Cross-Validation Accuracy Distribution')
plt.xticks([1], ['5-Fold CV'])
plt.grid(True, alpha=0.3)
plt.show()

---

## Challenge Problems (Optional)

### Challenge 1: Implement Gini Impurity from Scratch

Implement the Gini impurity calculation and compare with sklearn.

In [ ]:
def gini_impurity(y):
    """
    Calculate Gini impurity.
    
    Gini = 1 - Σ(p_i²)
    
    Parameters:
    -----------
    y : array-like
        Target values
    
    Returns:
    --------
    float
        Gini impurity (0 = pure, higher = more impure)
    """
    # Your code here
    pass


# Test
print("Testing Gini Impurity:")
print(f"Gini([0,0,0,0]): {gini_impurity([0,0,0,0]):.4f} (should be 0.0000)")
print(f"Gini([0,0,1,1]): {gini_impurity([0,0,1,1]):.4f} (should be 0.5000)")
print(f"Gini([0,1,2]): {gini_impurity([0,1,2]):.4f} (should be 0.6667)")

### Challenge 2: Implement Pruning

Add post-pruning to your ID3 implementation to reduce overfitting.

In [ ]:
def prune_tree(node, X_val, y_val):
    """
    Prune a decision tree using validation data.
    
    Strategy: Reduced Error Pruning
    - Try replacing each internal node with a leaf
    - Keep the change if validation accuracy improves or stays the same
    
    Parameters:
    -----------
    node : ID3Node
        Root of tree/subtree to prune
    X_val : validation features
    y_val : validation labels
    
    Returns:
    --------
    ID3Node
        Pruned tree
    """
    # Your code here
    # This is a challenging problem!
    pass

print("Challenge: Implement tree pruning!")

### Challenge 3: Implement Cost-Complexity Pruning

Implement CART's cost-complexity pruning algorithm.

In [ ]:
# Use sklearn's built-in cost-complexity pruning
# Your task: Understand and visualize the trade-off

# Get cost-complexity pruning path
path = cart_cancer.cost_complexity_pruning_path(X_train_cancer, y_train_cancer)
ccp_alphas = path.ccp_alphas
impurities = path.impurities

# Train trees with different alpha values
trees = []
for alpha in ccp_alphas:
    tree = DecisionTreeClassifier(random_state=42, ccp_alpha=alpha)
    tree.fit(X_train_cancer, y_train_cancer)
    trees.append(tree)

# Evaluate
train_scores = [tree.score(X_train_cancer, y_train_cancer) for tree in trees]
test_scores = [tree.score(X_test_cancer, y_test_cancer) for tree in trees]

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(ccp_alphas, train_scores, marker='o', label='Train', drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='s', label='Test', drawstyle="steps-post")
ax.set_xlabel('Alpha (complexity parameter)')
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy vs Alpha for Cost-Complexity Pruning')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

# Find best alpha
best_idx = np.argmax(test_scores)
print(f"\nBest alpha: {ccp_alphas[best_idx]:.6f}")
print(f"Test accuracy: {test_scores[best_idx]:.4f}")
print(f"Tree depth: {trees[best_idx].get_depth()}")
print(f"Number of leaves: {trees[best_idx].get_n_leaves()}")

### Challenge 4: Multi-way Split vs Binary Split Comparison

Compare multi-way splits (ID3) with binary splits (CART) on a categorical dataset.

In [ ]:
# Create a dataset with a multi-valued categorical feature
np.random.seed(42)
n_samples = 200

# Feature with 5 categories
categories = ['A', 'B', 'C', 'D', 'E']
X_multi = np.random.choice(categories, size=(n_samples, 1))

# Target correlated with categories
# A, B → Class 0; C, D, E → Class 1
y_multi = np.array([0 if x[0] in ['A', 'B'] else 1 for x in X_multi])
y_multi = (y_multi + np.random.binomial(1, 0.1, n_samples)) % 2  # Add 10% noise

# Convert for ID3
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_multi_encoded = le.fit_transform(X_multi.ravel()).reshape(-1, 1)

# ID3: Will create 5-way split
id3_multi = ID3Classifier(max_depth=3)
id3_multi.fit(X_multi_encoded, y_multi, feature_names=['Category'])

print("ID3 Tree (Multi-way split):")
print("=" * 40)
id3_multi.print_tree()
print("=" * 40)

# CART: Will create binary splits
cart_multi = DecisionTreeClassifier(max_depth=3, random_state=42)
cart_multi.fit(X_multi_encoded, y_multi)

print("\nCART Tree (Binary splits):")
plt.figure(figsize=(12, 8))
plot_tree(cart_multi, feature_names=['Category'], class_names=['Class 0', 'Class 1'],
          filled=True, rounded=True)
plt.title('CART with Binary Splits')
plt.show()

print("\nObservation:")
print("- ID3 splits into 5 branches (one per category value)")
print("- CART creates binary splits (Category <= threshold)")
print("- For categorical data, ID3's multi-way splits can be more intuitive")
print("- CART may need multiple binary splits to achieve same result")

---

## Reflection Questions

1. **When should you use ID3 vs CART?**
   - Consider data types, interpretability, and computational efficiency

2. **Why does C4.5 use Gain Ratio instead of Information Gain?**
   - Think about attributes with many unique values (like IDs)

3. **What are the advantages of binary splits (CART) over multi-way splits (ID3)?**
   - Consider tree balance, implementation simplicity

4. **How does discretization affect ID3's performance?**
   - What information is lost? What are the trade-offs?

5. **When would Gini impurity give different results than Entropy?**
   - They're similar but not identical - when does the difference matter?

6. **How would you handle a dataset with both categorical and continuous features?**
   - What preprocessing is needed for ID3 vs CART?

7. **Why is pruning important?**
   - What happens without pruning on noisy data?

8. **In medical diagnosis (like breast cancer), which type of error is worse?**
   - False positive (predict malignant when benign)?
   - False negative (predict benign when malignant)?
   - How would you adjust your model?

---

## Summary

In this exercise, you learned:

✓ **ID3 Algorithm**: Information Gain-based tree construction for categorical data  
✓ **C4.5 Improvements**: Gain Ratio to handle multi-valued attributes  
✓ **CART Algorithm**: Gini-based binary splitting with native continuous feature support  
✓ **Discretization**: Converting continuous features for categorical algorithms  
✓ **Missing Value Handling**: Imputation strategies and their impact  
✓ **Feature Importance**: Understanding which features drive decisions  
✓ **Real-World Application**: Medical diagnosis with breast cancer dataset  
✓ **Algorithm Comparison**: Trade-offs between ID3, C4.5, and CART  
✓ **Pruning Concepts**: Reducing overfitting with cost-complexity pruning  

**Key Takeaways:**

| Algorithm | Criterion | Splits | Continuous Features | Best For |
|-----------|-----------|--------|---------------------|----------|
| ID3 | Information Gain | Multi-way | No (discretize) | Categorical data, interpretability |
| C4.5 | Gain Ratio | Multi-way | Yes (discretization) | Avoiding bias, mixed data |
| CART | Gini/Entropy | Binary | Yes (native) | General purpose, efficiency |

**Next Steps:**

- Review the [Decision Trees lesson](https://jumpingsphinx.github.io/module3-trees/01-decision-trees/)
- Review the [Tree Algorithms lesson](https://jumpingsphinx.github.io/module3-trees/02-tree-algorithms/)
- Complete Exercise 3 on Ensemble Methods (Random Forests, Boosting)
- Try implementing a regression tree (CART for regression)

---

**Need help?** Check the solution notebook or open an issue on [GitHub](https://github.com/jumpingsphinx/jumpingsphinx.github.io/issues).